# Data preparation for prediction

The task for the prediction is as follows:

... forecasting total system-level demand in the next hour ...

Therefore we need a dataset which has an hourly resolution. An intuitive system-level demand measure would be the amount of trips per hour. To achive this we resample the whole dataset by the hour and count the number of trips per hour.

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

In [3]:
#read dataset
path = "../adrian_workspace/chicago_2017_REDUCED.csv"
bike_data = pd.read_csv(path, parse_dates=["start_time","end_time"], index_col=0)
bike_data.head(1)

,start_time,end_time,start_station_id,end_station_id,bike_id,user_type,start_hour,start_day,start_month,start_weekday,max_temp,min_temp,precip,start_long,start_lat,end_lat,end_long
6,2017-01-01 00:11:34,2017-01-01 00:23:47,194,68,1145,Subscriber,0,1,1,6,-0.6,-0.6,0.0,-87.62603,41.886875,41.875885,-87.640795


The data contains trips specfic information like the start and end station. Since this data cannot be used for system-level demand those features are not needed for the prediction.

In [4]:
# feature engineering
# add trip duration feature
timedelta = bike_data["end_time"] - bike_data["start_time"]
bike_data["duration"] = timedelta.apply(lambda x:x.seconds)
bike_data["duration_min"] = bike_data["duration"].apply(lambda x:(int) (x/60))
# add weekend dummy variable
bike_data["is_weekend"] = bike_data["start_weekday"].isin([5,6])
# only include possibel features
feature_data = bike_data[["start_time","bike_id","start_hour","start_day","start_month","start_weekday","is_weekend","duration_min","max_temp","precip"]]
feature_data.head(1)


,start_time,bike_id,start_hour,start_day,start_month,start_weekday,is_weekend,duration_min,max_temp,precip
6,2017-01-01 00:11:34,1145,0,1,1,6,True,12,-0.6,0.0


In [5]:
#aggregate by hour (for demand prediction in the next hour)
feature_data_by_hour = feature_data.groupby(pd.Grouper(key='start_time',freq='1H')).agg({"bike_id":"count","start_hour":"mean","start_day":"mean","start_month":"mean","start_weekday":"mean","is_weekend":"mean","duration_min":"mean","max_temp":"mean","precip":"mean"})
feature_data_by_hour.rename(columns={"duration_min":"mean_duration_min","bike_id":"trip_amount"}, inplace=True)
feature_data_by_hour.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2017-01-01 00:00:00 to 2017-12-31 23:00:00
Freq: H
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   trip_amount        8760 non-null   int64  
 1   start_hour         8237 non-null   float64
 2   start_day          8237 non-null   float64
 3   start_month        8237 non-null   float64
 4   start_weekday      8237 non-null   float64
 5   is_weekend         8237 non-null   float64
 6   mean_duration_min  8237 non-null   float64
 7   max_temp           8237 non-null   float64
 8   precip             8237 non-null   float64
dtypes: float64(8), int64(1)
memory usage: 684.4 KB


As we can see there are some hours where there were no trips done. This results in null values. However we predictive models can not handle null values. To make the data usable we forwards fill the weather data and calculate values for the other columns.

In [19]:
# deal with NaN values
feature_data_by_hour.sort_index(inplace=True)
feature_data_by_hour["max_temp"] = feature_data_by_hour["max_temp"].ffill()
feature_data_by_hour["precip"] = feature_data_by_hour["precip"].ffill()
feature_data_by_hour["mean_duration_min"] = feature_data_by_hour["mean_duration_min"].fillna(value=0)
feature_data_by_hour["start_hour"] = feature_data_by_hour.index.to_series().apply(lambda x:x.hour)
feature_data_by_hour["start_day"] = feature_data_by_hour.index.to_series().apply(lambda x:x.day)
feature_data_by_hour["start_month"] = feature_data_by_hour.index.to_series().apply(lambda x:x.month)
feature_data_by_hour["start_weekday"] = feature_data_by_hour.index.to_series().apply(lambda x:x.dayofweek)
feature_data_by_hour["is_weekend"] = feature_data_by_hour["start_weekday"].apply(lambda x: 1 if (x in [5,6]) else 0 )
feature_data_by_hour.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2017-01-01 00:00:00 to 2017-12-31 23:00:00
Freq: H
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   trip_amount        8760 non-null   int64  
 1   start_hour         8760 non-null   int64  
 2   start_day          8760 non-null   int64  
 3   start_month        8760 non-null   int64  
 4   start_weekday      8760 non-null   int64  
 5   is_weekend         8760 non-null   int64  
 6   mean_duration_min  8760 non-null   float64
 7   max_temp           8760 non-null   float64
 8   precip             8760 non-null   float64
dtypes: float64(3), int64(6)
memory usage: 684.4 KB


In [20]:
feature_data_by_hour.head()

,trip_amount,start_hour,start_day,start_month,start_weekday,is_weekend,mean_duration_min,max_temp,precip
start_time,,,,,,,,,
2017-01-01 00:00:00,4,0,1,1,6,1,9.5,-0.6,0.0
2017-01-01 01:00:00,4,1,1,1,6,1,10.0,-2.2,0.0
2017-01-01 02:00:00,3,2,1,1,6,1,4.0,-2.8,0.0
2017-01-01 03:00:00,1,3,1,1,6,1,10.0,-3.3,0.0
2017-01-01 04:00:00,0,4,1,1,6,1,0.0,-3.3,0.0


Now we have a good dataset for creating ML models.

In [21]:
# save data as csv
# feature_data_by_hour.to_csv("load_prediction_data.csv")